# Recommender Systems 2020/21

## Practice session on MF recommenders.

### Outline
* MF Recommenders
* BPR-MF
* PureSVD
* Comparison between MF recommenders


## Administrative code

Download the dataset and generate _TRAIN_ and _TEST_ splits

In [1]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

data_reader = Movielens10MReader()
datasets_dict = data_reader.load_data()


Movielens10M: Verifying data consistency...
Movielens10M: Verifying data consistency... Passed!
DataReader: current dataset is: <class 'Data_manager.Dataset.Dataset'>
	Number of items: 10681
	Number of users: 69878
	Number of interactions in URM_all: 10000054
	Value range in URM_all: 0.50-5.00
	Interaction density: 1.34E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.43E+02
		 Max: 7.36E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 9.36E+02
		 Max: 3.49E+04
	Gini Index: 0.57

	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 20, feature occurrences: 21564, density 1.01E-01
	ICM name: ICM_tags, Value range: 1.00 / 69.00, Num features: 10217, feature occurrences: 108563, density 9.95E-04
	ICM name: ICM_all, Value range: 1.00 / 69.00, Num features: 10237, feature occurrences: 130127, density 1.19E-03




In [2]:
URM_all = datasets_dict.AVAILABLE_URM["URM_all"]
print(URM_all)

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

  (0, 0)	5.0
  (0, 1)	5.0
  (0, 2)	5.0
  (0, 3)	5.0
  (0, 4)	5.0
  (0, 5)	5.0
  (0, 6)	5.0
  (0, 7)	5.0
  (0, 8)	5.0
  (0, 9)	5.0
  (0, 10)	5.0
  (0, 11)	5.0
  (0, 12)	5.0
  (0, 13)	5.0
  (0, 14)	5.0
  (0, 15)	5.0
  (0, 16)	5.0
  (0, 17)	5.0
  (0, 18)	5.0
  (0, 19)	5.0
  (0, 20)	5.0
  (0, 21)	5.0
  (1, 16)	3.0
  (1, 22)	5.0
  (1, 23)	3.0
  :	:
  (69877, 463)	3.0
  (69877, 467)	1.0
  (69877, 468)	4.0
  (69877, 475)	2.0
  (69877, 481)	3.0
  (69877, 486)	4.0
  (69877, 505)	3.0
  (69877, 518)	1.0
  (69877, 537)	5.0
  (69877, 541)	2.0
  (69877, 1081)	2.0
  (69877, 1302)	4.0
  (69877, 1322)	2.0
  (69877, 1436)	4.0
  (69877, 1609)	1.0
  (69877, 1646)	3.0
  (69877, 1660)	2.0
  (69877, 1671)	2.0
  (69877, 2001)	4.0
  (69877, 2065)	1.0
  (69877, 2941)	1.0
  (69877, 3066)	1.0
  (69877, 3386)	3.0
  (69877, 3448)	1.0
  (69877, 5330)	1.0


## MF Computing prediction

<img src="https://miro.medium.com/max/988/1*tiF4e4Y-wVH732_6TbJVmQ.png" alt="Latent Factors" style="margin: 0 auto;"/>

In a MF model you have two matrices, let's call them $U$ and $V$. 

In $U$ you have one row for every user (i.e., users are in the rows). In $V$ you have one column for every item (i.e., items are in the columns). The other dimension (columns for $U$ and rows for $V$) is called latent factors.



<img src="https://miro.medium.com/max/988/1*tiF4e4Y-wVH732_6TbJVmQ.png" alt="Latent Factors" style="margin: 0 auto;"/>


The number of latent factors is variable (a hyperparameter), if we represent the number of item factors by $d$, then $U$ is an $m \times d$ matrix and $V$ is a $d \times n$ matrix. Lastly, $U$ is called the user factors matrix and $V$ is the item factors matrix.

In [3]:
num_factors = 10
num_users, num_items = URM_train.shape

num_factors, num_users, num_items

(10, 69878, 10681)

In [4]:
import numpy as np

# user_factors is our U, U is n_users x num_factors
user_factors = np.random.random((num_users, num_factors))

# item_factors is our V
item_factors = np.random.random((num_items, num_factors))

In [5]:
user_factors, user_factors.shape

(array([[0.77105129, 0.26981857, 0.21525026, ..., 0.81810914, 0.18029247,
         0.97124718],
        [0.73256688, 0.19107128, 0.0166121 , ..., 0.93479036, 0.27351954,
         0.82409332],
        [0.01614075, 0.59567967, 0.00828774, ..., 0.00915704, 0.81327579,
         0.73391817],
        ...,
        [0.50609718, 0.51421094, 0.53414818, ..., 0.19367252, 0.89929362,
         0.72968165],
        [0.66306608, 0.50086656, 0.2686102 , ..., 0.71329147, 0.98767928,
         0.09459023],
        [0.38507525, 0.95404299, 0.62014664, ..., 0.67315072, 0.02186156,
         0.35256418]]),
 (69878, 10))

In [6]:
item_factors, item_factors.shape

(array([[0.34736724, 0.15438443, 0.68194495, ..., 0.16829192, 0.31393316,
         0.32430294],
        [0.68942466, 0.78205893, 0.10659545, ..., 0.3056192 , 0.48205477,
         0.12945008],
        [0.20601237, 0.33132837, 0.58948576, ..., 0.73873408, 0.73419341,
         0.63150557],
        ...,
        [0.17463286, 0.36053267, 0.77677085, ..., 0.93123106, 0.55821074,
         0.8863073 ],
        [0.65169201, 0.71092907, 0.32342315, ..., 0.67654419, 0.33397039,
         0.68935113],
        [0.9926887 , 0.13838396, 0.63449694, ..., 0.98416582, 0.88046898,
         0.10438477]]),
 (10681, 10))

To compute the prediction we have to muliply the user factors to the item factors

$$\hat{x}_{ui} = \langle w_u,h_i \rangle = \sum_{f=1}^k w_{uf} \cdot h_{if}$$



### Zooming-in in an specific user.

Let's suppose Bob is user 42.

<img src="images/bob_example.jpeg" style="margin: 0 auto;width: 250px;" alt="Bob Image">

And that we're interested into the movie Toy Story (with id 15).

<img src="images/toy_story_poster.png" style="margin: 0 auto;width: 250px;" alt="Toy Story Poster Image">

In our specific case, let's see what is the rating for user $u = 42$ and item $i = 15$

In [7]:
item_index = 15
user_index = 42

prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])

print("Prediction is {:.2f}".format(prediction))

Prediction is 1.51


## MF BPR models

### Recap on BPR
S.Rendle et al. BPR: Bayesian Personalized Ranking from Implicit Feedback. UAI2009

The usual approach for item recommenders is to predict a personalized score $\hat{x}_{ui}$ for an item that reflects the preference of the user for the item. Then the items are ranked by sorting them according to that score.

Machine learning approaches are tipically fit by using observed items as a positive sample and missing ones for the negative class. A perfect model would thus be useless, as it would classify as negative (non-interesting) all the items that were non-observed at training time. The only reason why such methods work is regularization.

BPR use a different approach. The training dataset is composed by triplets $(u,i,j)$ representing that user u is assumed to prefer i over j. For an implicit dataset this means that u observed i but not j:
$$D_S := \{(u,i,j) \mid i \in I_u^+ \wedge j \in I \setminus I_u^+\}$$


### BPR-OPT
A machine learning model can be represented by a parameter vector $\Theta$ which is found at fitting time. BPR wants to find the parameter vector that is most probable given the desired, but latent, preference structure $>_u$:
$$p(\Theta \mid >_u) \propto p(>_u \mid \Theta)p(\Theta) $$
$$\prod_{u\in U} p(>_u \mid \Theta) = \dots = \prod_{(u,i,j) \in D_S} p(i >_u j \mid \Theta) $$

The probability that a user really prefers item $i$ to item $j$ is defined as:
$$ p(i >_u j \mid \Theta) := \sigma(\hat{x}_{uij}(\Theta)) $$
Where $\sigma$ represent the logistic sigmoid and $\hat{x}_{uij}(\Theta)$ is an arbitrary real-valued function of $\Theta$ (the output of your arbitrary model).



To complete the Bayesian setting, we define a prior density for the parameters:
$$p(\Theta) \sim N(0, \Sigma_\Theta)$$
And we can now formulate the maximum posterior estimator:
$$BPR-OPT := \log p(\Theta \mid >_u) $$
$$ = \log p(>_u \mid \Theta) p(\Theta) $$
$$ = \log \prod_{(u,i,j) \in D_S} \sigma(\hat{x}_{uij})p(\Theta) $$
$$ = \sum_{(u,i,j) \in D_S} \log \sigma(\hat{x}_{uij}) + \log p(\Theta) $$
$$ = \sum_{(u,i,j) \in D_S} \log \sigma(\hat{x}_{uij}) - \lambda_\Theta ||\Theta||^2 $$

Where $\lambda_\Theta$ are model specific regularization parameters.



### BPR learning algorithm
Once obtained the log-likelihood, we need to maximize it in order to find our obtimal $\Theta$. As the criterion is differentiable, gradient descent algorithms are an obvious choiche for maximization.

Gradient descent comes in many fashions, you can find an overview on this thesis https://www.politesi.polimi.it/bitstream/10589/133864/3/tesi.pdf on pages 18-19-20. A nice post about momentum is available here https://distill.pub/2017/momentum/

The basic version of gradient descent consists in evaluating the gradient using all the available samples and then perform a single update. The problem with this is, in our case, that our training dataset is very skewed. Suppose an item $i$ is very popular. Then we have many terms of the form $\hat{x}_{uij}$ in the loss because for many users u the item $i$ is compared against all negative items $j$.


The other popular approach is stochastic gradient descent, where for each training sample an update is performed. This is a better approach, but the order in which the samples are traversed is crucial. To solve this issue BPR uses a stochastic gradient descent algorithm that choses the triples randomly.

The gradient of BPR-OPT with respect to the model parameters is: 
$$\frac{\partial BPR-OPT}{\partial \Theta} = \sum_{(u,i,j) \in D_S} \frac{\partial}{\partial \Theta} \log \sigma (\hat{x}_{uij}) - \lambda_\Theta \frac{\partial}{\partial\Theta} || \Theta ||^2$$
$$ =  \sum_{(u,i,j) \in D_S} \frac{-e^{-\hat{x}_{uij}}}{1+e^{-\hat{x}_{uij}}} \frac{\partial}{\partial \Theta}\hat{x}_{uij} - \lambda_\Theta \Theta $$



### BPR-MF

In order to practically apply this learning schema to an existing algorithm, we first split the real valued preference term: $\hat{x}_{uij} := \hat{x}_{ui} − \hat{x}_{uj}$. And now we can apply any standard collaborative filtering model that predicts $\hat{x}_{ui}$.

The problem of predicting $\hat{x}_{ui}$ can be seen as the task of estimating a matrix $X:U×I$. With matrix factorization the target matrix $X$ is approximated by the matrix product of two low-rank matrices $W:|U|\times k$ and $H:|I|\times k$:
$$X := WH^t$$
The prediction formula can also be written as:
$$\hat{x}_{ui} = \langle w_u,h_i \rangle = \sum_{f=1}^k w_{uf} \cdot h_{if}$$
Besides the dot product ⟨⋅,⋅⟩, in general any kernel can be used.



We can now specify the derivatives:
$$ \frac{\partial}{\partial \theta} \hat{x}_{uij} = \begin{cases}
(h_{if} - h_{jf}) \text{ if } \theta=w_{uf}, \\
w_{uf} \text{ if } \theta = h_{if}, \\
-w_{uf} \text{ if } \theta = h_{jf}, \\
0 \text{ else }
\end{cases} $$

Which basically means: user $u$ prefer $i$ over $j$, let's do the following:
- Increase the relevance (according to $u$) of features belonging to $i$ but not to $j$ and vice-versa
- Increase the relevance of features assigned to $i$
- Decrease the relevance of features assigned to $j$




## Train a MF MSE model

### Use SGD as we saw for SLIM

Keeping the same idea for $u = 42$ and $i = 15$, let's suppose that the real rating $r_{u,i} = 5$

In [8]:
test_data = 5

prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])
gradient = test_data - prediction

print(f"* Prediction error is {gradient:.2f}")
print(f"* Real value (r_u,i) is {test_data}")
print(f"* Prediction (rhat_u,i) is {prediction:.2f}")

* Prediction error is 3.49
* Real value (r_u,i) is 5
* Prediction (rhat_u,i) is 1.51


Remember, for SGD we have a regularization parameter $\lambda_\Theta$ and the learning rate hyperparameter $lr$ 

In [9]:
learning_rate = 1e-2
regularization = 1e-3

# Copy original value to avoid messing up the updates
H_i = item_factors[item_index,:]
W_u = user_factors[user_index,:]

user_factors[user_index,:] += learning_rate * (gradient * H_i - regularization * W_u)
item_factors[item_index,:] += learning_rate * (gradient * W_u - regularization * H_i)

Now that we updated the user and item factors, let's see the new prediction

In [10]:
new_prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])
new_gradient = test_data - new_prediction
 
print(f"* Prediction error is {new_gradient:.2f}")
print(f"* Real value (r_u,i) is {test_data}")
print(f"* Prediction (rhat_u,i) is {new_prediction:.2f}")

* Prediction error is 3.30
* Real value (r_u,i) is 5
* Prediction (rhat_u,i) is 1.70


Just for comparison

In [11]:
print(f"Feature\t\t\t|Previous Values \t|After Gradient values")
print(f"___________________________________________________________________________")
print(f"Prediction Error\t|{gradient:.2f}\t\t\t|{new_gradient:.2f}")
print(f"Real value (r_u,i)\t|{test_data}\t\t\t|{test_data}")
print(f"Prediction (rhat_u,i)\t|{prediction:.2f}\t\t\t|{new_prediction:.2f}")

Feature			|Previous Values 	|After Gradient values
___________________________________________________________________________
Prediction Error	|3.49			|3.30
Real value (r_u,i)	|5			|5
Prediction (rhat_u,i)	|1.51			|1.70


### ⚠️WARNING: Initialization must be done with random non-zero values⚠️

... otherwise this happens

In [12]:
user_factors = np.zeros((num_users, num_factors))
item_factors = np.zeros((num_items, num_factors))

In [13]:
prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])
gradient = test_data - prediction

print(f"* Prediction error is {gradient:.2f}")
print(f"* Real value (r_u,i) is {test_data}")
print(f"* Prediction (rhat_u,i) is {prediction}")

* Prediction error is 5.00
* Real value (r_u,i) is 5
* Prediction (rhat_u,i) is 0.0


In [14]:
W_u = user_factors[user_index,:]
H_i = item_factors[item_index,:]

user_factors[user_index,:] += learning_rate * (gradient * H_i - regularization * W_u)
item_factors[item_index,:] += learning_rate * (gradient * W_u - regularization * H_i)

In [15]:
new_prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])
new_gradient = test_data - new_prediction

print(f"* Prediction error is {new_gradient:.2f}")
print(f"* Real value (r_u,i) is {test_data}")
print(f"* Prediction (rhat_u,i) is {new_prediction}")

* Prediction error is 5.00
* Real value (r_u,i) is 5
* Prediction (rhat_u,i) is 0.0


### Since the updates multiply the gradient and the latent factors, if those are zero the SGD will never be able to move from that point

## Now with BPR MF

The basics are the same, except for how we compute the gradient, where we have to sample triplets $(u, i^+, i^-)$ instead.

In [16]:
URM_mask = URM_train.copy()
URM_mask.data[URM_mask.data <= 3] = 0

URM_mask.eliminate_zeros()

# Extract users having at least one interaction to choose from
eligibleUsers = []

for user_id in range(num_users):
    start_pos = URM_mask.indptr[user_id]
    end_pos = URM_mask.indptr[user_id+1]

    if len(URM_mask.indices[start_pos:end_pos]) > 0:
        eligibleUsers.append(user_id)  
        
num_users, len(eligibleUsers)

(69878, 69802)

In [17]:
def sampleTriplet():
    # By randomly selecting a user in this way we could end up 
    # with a user with no interactions
    #user_id = np.random.randint(0, n_users)
    user_id = np.random.choice(eligibleUsers)
    
    # Get user seen items and choose one
    userSeenItems = URM_mask[user_id,:].indices
    pos_item_id = np.random.choice(userSeenItems)

    negItemSelected = False

    # It's faster to just try again than to build a mapping of the non-seen items
    while (not negItemSelected):
        neg_item_id = np.random.randint(0, num_items)

        if (neg_item_id not in userSeenItems):
            negItemSelected = True

    return user_id, pos_item_id, neg_item_id

In [18]:
for _ in range(10):
    print(sampleTriplet())

(61434, 1059, 3727)
(57395, 266, 5167)
(29218, 1371, 3484)
(6548, 137, 1429)
(51378, 391, 4220)
(21840, 584, 4551)
(1986, 1089, 543)
(62562, 1611, 3028)
(58025, 327, 7243)
(2145, 1077, 5364)


In [19]:
user_factors = np.random.random((num_users, num_factors))
item_factors = np.random.random((num_items, num_factors))

In [20]:
user_id, positive_item, negative_item = sampleTriplet()

print(user_id, positive_item, negative_item)

2518 864 145


In [21]:
user_factors[user_id, :], item_factors[positive_item,:], item_factors[negative_item,:]

(array([0.25018247, 0.45896203, 0.66343316, 0.1443068 , 0.27183938,
        0.56821441, 0.39703548, 0.44701106, 0.15537737, 0.3435701 ]),
 array([0.03223317, 0.80434417, 0.89772161, 0.81113855, 0.4956003 ,
        0.85430307, 0.64105046, 0.34943407, 0.08350337, 0.42093353]),
 array([0.51465736, 0.85777222, 0.33279313, 0.58213778, 0.51465036,
        0.87294734, 0.49067511, 0.92394343, 0.96496654, 0.26542339]))

In [22]:
x_uij = np.dot(user_factors[user_id, :], 
               (item_factors[positive_item,:] - item_factors[negative_item,:]))
x_uij 

-0.03378784701867701

In [23]:
sigmoid_item = 1 / (1 + np.exp(x_uij))
sigmoid_item

0.5084461582456739

#### When using BPR we have to update three components, the user factors and the item factors of both the positive and negative item

In [24]:
H_i = item_factors[positive_item,:]
H_j = item_factors[negative_item,:]
W_u = user_factors[user_id,:]

user_factors[user_index,:] += learning_rate * (sigmoid_item * ( H_i - H_j ) - regularization * W_u)
item_factors[positive_item,:] += learning_rate * (sigmoid_item * ( W_u ) - regularization * H_i)
item_factors[negative_item,:] += learning_rate * (sigmoid_item * (-W_u ) - regularization * H_j)

In [25]:
new_x_uij = np.dot(user_factors[user_id, :], (item_factors[positive_item,:] - item_factors[negative_item,:]))
new_x_uij

-0.017205978726479706

For comparison

In [26]:
print(f"Feature\t\t\t|Previous Values \t|After Gradient values")
print(f"___________________________________________________________________________")
print(f"X_uij\t\t\t|{x_uij:.5f}\t\t|{new_x_uij:.5f}")


Feature			|Previous Values 	|After Gradient values
___________________________________________________________________________
X_uij			|-0.03379		|-0.01721


### How to rank items with MF ?

Compute the prediction for all items and rank them

In [27]:
item_scores = np.dot(user_factors[user_index,:], item_factors.T)
item_scores

array([2.88613433, 3.26810738, 2.23810243, ..., 2.97465278, 2.98534907,
       2.7113013 ])

In [28]:
item_scores.shape

(10681,)

### Early stopping, how to use it and when it is needed

Problem, how many epochs? 5, 10, 150, 2487 ?

We could try different values in increasing order: 5, 10, 15, 20, 25...

### However, in this way we would train up to a point, test and then discard the model, to re-train it again up to that same point and then some more... not a good idea.


### Early stopping! 
* Train the model up to a certain number of epochs, say 5
* Compute the recommendation quality on the validation set
* Train for other 5 epochs
* Compute the recommendation quality on the validation set AND compare it with the previous one. If better, then we have another best model, if not, go ahead...
* Repeat until you have either reached the max number of epoch you want to allow (e.g., 300) or a certain number of contiguous validation seps have not updated te best model


### Advantages:
* Easy to implement, we already have all that is required, a train function, a predictor function and an evaluator
* MUCH faster than retraining everything from the beginning
* Often allows to reach even better solutions

### Challenges:
* The evaluation step may be very slow compared to the time it takes to re-train the model

## PureSVD model

### As opposed to the previous ones, PureSVD relies on the SVD decomposition of the URM, which is an easily available function

In our case, an SVD decomposition of the URM ($m \times n$)is as follows

$$ URM = U \Sigma V^T $$

Where $U$ is an orthogonal $m \times m$ matrix, $\Sigma$ is a rectangular diagonal matrix ($m \times n$), and $V^T$ is an orthogonal $n \times n$ matrix. 



However, calculating the SVD for the whole URM consumes lots of resources (time and memory), we can use a *truncated* version of SVD called *Truncated SVD*, the idea is similar to the original SVD, but instead of calculating an *exact* decomposition, we approximate URM:

$$ \widehat{URM} = U_{t} \Sigma_{t} V^*_{t} $$

Where $U_{t}$ is a $m \times t$ matrix, $\Sigma_{t}$ is a $t$ vector, and $V^*_{t}$ is a $t \times n$ matrix. For this approximation, only the $t$ largest singular values are kept.

In [29]:
from sklearn.utils.extmath import randomized_svd

# Other SVDs are also available, like from sklearn.decomposition import TruncatedSVD

In [30]:
U, Sigma, VT = randomized_svd(URM_train,
                              n_components=num_factors,
                              random_state=1234)

In [31]:
U, U.shape

(array([[ 7.39053964e-04, -2.93912628e-03, -7.78126692e-04, ...,
         -7.57091607e-04, -2.20537323e-03, -6.29395109e-04],
        [ 6.10583480e-04, -1.40230446e-03, -1.27487887e-04, ...,
          3.09211577e-03,  2.39981610e-03,  6.87632605e-04],
        [ 5.50076917e-04,  2.22152832e-04, -5.19179339e-04, ...,
          6.24366494e-05,  1.50688570e-03,  3.57234614e-04],
        ...,
        [ 3.42185192e-03,  1.54442175e-03,  5.47862095e-03, ...,
         -1.72592588e-03,  2.38431771e-03,  3.72140356e-03],
        [ 1.24815540e-03, -4.66883175e-03,  1.04187522e-03, ...,
          8.30097296e-04, -2.00077559e-03,  1.99906293e-03],
        [ 1.42310674e-03, -7.13603494e-04, -8.82303566e-04, ...,
          2.30381742e-03,  1.03186639e-04,  4.95088268e-03]]),
 (69878, 10))

In [32]:
Sigma, Sigma.shape

(array([4276.64643802, 1784.31557975, 1533.5025626 , 1226.93392479,
        1185.4530811 , 1014.45064137,  960.6592356 ,  908.72211896,
         842.90214758,  745.34494295]),
 (10,))

In [33]:
VT, VT.shape

(array([[ 0.00665443,  0.03274073,  0.04184214, ...,  0.        ,
          0.        ,  0.        ],
        [-0.01478895, -0.09538786, -0.07150558, ..., -0.        ,
         -0.        , -0.        ],
        [ 0.00297177, -0.01156154, -0.04204534, ..., -0.        ,
         -0.        , -0.        ],
        ...,
        [ 0.00167712, -0.02233305, -0.03248025, ..., -0.        ,
         -0.        , -0.        ],
        [-0.0062943 , -0.00451829, -0.02969469, ...,  0.        ,
          0.        ,  0.        ],
        [-0.01288507,  0.01692335, -0.0836164 , ..., -0.        ,
         -0.        , -0.        ]]),
 (10, 10681))

In [34]:
VT.shape

(10, 10681)

### Computing a prediction

So, how do we transform the matrices $U_t$, $\Sigma_t$, $V^T_t$ into something that we can use for recommendation? 

Remember, $\widehat{URM} = U_t \Sigma_t V^T_t$.

#### Matrix Factorization Approach (PureSVDRecommender)

Consider the following matrices $W = U_t \Sigma_t$, and $H = V^T_t$, then we just have a Matrix Factorization recommender where $\widehat{URM} = WH$, where $W$ represents the user factors and $H$ are the item factors.

In [35]:
import scipy.sparse as sps

# Store an intermediate pre-multiplied matrix
user_factors = U * sps.diags(Sigma)
item_factors = VT

Let's now predict if Bob would like Toy Story or not...

In [36]:
prediction = user_factors[user_index, :].dot(item_factors[:,item_index])

print("Prediction is {:.2f}".format(prediction))

Prediction is -0.45


And with this we calculate the score of all items for Bob.

In [37]:
item_scores = user_factors[user_index, :].dot(item_factors)
item_scores

array([-0.19917472, -0.26403461, -1.02277717, ...,  0.        ,
        0.        ,  0.        ])

In [38]:
item_scores.shape

(10681,)

So, which are the best 20 items for Bob?

In [39]:
best_items_for_bob = np.flip(np.argsort(item_scores))[:20]
best_items_for_bob

array([145, 219, 170, 218, 148, 146, 166, 167, 133, 176,  44, 163, 144,
        34, 227, 220, 150, 147, 182, 193])

Is Toy story inside that list?

In [40]:
item_index in best_items_for_bob

False

#### Item-Based Approach (PureSVDItemRecommender)

Consider the following matrix $P = U_t \Sigma_t$.

As $U$ and $V^T$ are orthogonal (meaning $UU^T = U^TU = I$), then 

$$ \widehat{URM} = U_t \Sigma_t V^T_t $$
$$ \widehat{URM}V = U_t \Sigma_t V^T_t V $$
$$ \widehat{URM}V = U_t \Sigma_t $$

Re-arranging the equations

$$ P = U_t \Sigma_t = URMV $$

With this, if we define $r_u$ as the $u$-th row in the URM and $v^T_i$ as the $i$-th column in $V^T$ then we calculate any $\hat{r}_{u,i}$ as 

$$\hat{r}_{u,i} = r_u V v^T_i$$

Which is equivalent to having a similarity matrix.

In [41]:
# BEWARE: This consumes A LOT of memory
item_weights = np.dot(VT.T, VT)

In [42]:
%%time

ITEM_factors = VT.T
topK = 100

n_items, n_factors = ITEM_factors.shape

block_size = 100

start_item = 0
end_item = 0

values = []
rows = []
cols = []

# Compute all similarities for each item using vectorization
while start_item < n_items:

    end_item = min(n_items, start_item + block_size)

    this_block_weight = np.dot(ITEM_factors[start_item:end_item, :], ITEM_factors.T)

    for col_index_in_block in range(this_block_weight.shape[0]):

        this_column_weights = this_block_weight[col_index_in_block, :]
        item_original_index = start_item + col_index_in_block

        # Sort indices and select TopK
        # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
        # - Partition the data to extract the set of relevant items
        # - Sort only the relevant items
        # - Get the original item index
        relevant_items_partition = (-this_column_weights).argpartition(topK-1)[0:topK]
        relevant_items_partition_sorting = np.argsort(-this_column_weights[relevant_items_partition])
        top_k_idx = relevant_items_partition[relevant_items_partition_sorting]

        # Incrementally build sparse matrix, do not add zeros
        notZerosMask = this_column_weights[top_k_idx] != 0.0
        numNotZeros = np.sum(notZerosMask)

        values.extend(this_column_weights[top_k_idx][notZerosMask])
        rows.extend(top_k_idx[notZerosMask])
        cols.extend(np.ones(numNotZeros) * item_original_index)



    start_item += block_size

item_weights = sps.csr_matrix((values, (rows, cols)),
                          shape=(n_items, n_items),
                          dtype=np.float32)



CPU times: user 8.11 s, sys: 1.53 s, total: 9.64 s
Wall time: 2.88 s


In [43]:
item_weights, item_weights.shape 

(<10681x10681 sparse matrix of type '<class 'numpy.float32'>'
 	with 1065800 stored elements in Compressed Sparse Row format>,
 (10681, 10681))

In [44]:
item_scores = URM_train[user_index, :].dot(item_weights).A.flatten()
item_scores

array([0.03041735, 0.27881394, 0.32415883, ..., 0.        , 0.        ,
       0.        ])

In [45]:
item_scores.shape

(10681,)

So, which are the best 20 items for Bob?

In [46]:
best_items_for_bob = np.flip(np.argsort(item_scores))[:20]
best_items_for_bob

array([ 145,  179,   34,  605,  176,  146, 1293,  133,  219,  399,  177,
        218,   24,  165,  227,  148, 1008,  170,  166,  140])

Is Toy story inside that list?

In [47]:
item_index in best_items_for_bob

False

## Comparison: BPR, FunkSVD, PureSVD (MF and Item-Based)

In [48]:
from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender

from Base.Evaluation.Evaluator import EvaluatorHoldout

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[5, 20])

evaluator_validation_early_stopping = EvaluatorHoldout(URM_train, cutoff_list=[5], exclude_seen = False)


In [49]:
%%time

recommender = PureSVDRecommender(URM_train)
recommender.fit()

result_dict_puresvd, _ = evaluator_test.evaluateRecommender(recommender)

PureSVDRecommender: URM Detected 23 (0.22 %) cold items.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 40000 ( 57.31% ) in 30.66 sec. Users per second: 1304
EvaluatorHoldout: Processed 69799 ( 100.00% ) in 51.83 sec. Users per second: 1347
CPU times: user 1min 41s, sys: 9.66 s, total: 1min 50s
Wall time: 57.7 s


In [50]:
result_dict_puresvd

{5: {'ROC_AUC': 0.48879759499897085,
  'PRECISION': 0.3588188942535811,
  'PRECISION_RECALL_MIN_DEN': 0.3659947372694643,
  'RECALL': 0.11236172045405454,
  'MAP': 0.2902881607664011,
  'MRR': 0.5734380148712748,
  'NDCG': 0.16199299282251783,
  'F1': 0.1711339857000296,
  'HIT_RATE': 1.7940944712674967,
  'ARHR': 0.9040587974039381,
  'NOVELTY': 0.004571292997686143,
  'AVERAGE_POPULARITY': 0.3557680500196552,
  'DIVERSITY_MEAN_INTER_LIST': 0.978365772661157,
  'DIVERSITY_HERFINDAHL': 0.9956703511517455,
  'COVERAGE_ITEM': 0.08023593296507818,
  'COVERAGE_ITEM_CORRECT': 0.07752083138282932,
  'COVERAGE_USER': 0.9988694581985746,
  'COVERAGE_USER_CORRECT': 0.7546581184349866,
  'DIVERSITY_GINI': 0.021081071480885498,
  'SHANNON_ENTROPY': 8.245711062066341},
 20: {'ROC_AUC': 0.5844348823143795,
  'PRECISION': 0.2398085932463046,
  'PRECISION_RECALL_MIN_DEN': 0.34766106830383015,
  'RECALL': 0.25745904911809864,
  'MAP': 0.20132722886069132,
  'MRR': 0.5887065958768947,
  'NDCG': 0.25275

In [51]:
%%time
recommender = PureSVDItemRecommender(URM_train)
recommender.fit()

result_dict_puresvditem, _ = evaluator_test.evaluateRecommender(recommender)

PureSVDItemRecommender: URM Detected 23 (0.22 %) cold items.
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 9000 ( 12.89% ) in 32.40 sec. Users per second: 278
EvaluatorHoldout: Processed 19000 ( 27.22% ) in 1.08 min. Users per second: 292
EvaluatorHoldout: Processed 27000 ( 38.68% ) in 1.59 min. Users per second: 284
EvaluatorHoldout: Processed 36000 ( 51.58% ) in 2.15 min. Users per second: 280
EvaluatorHoldout: Processed 44000 ( 63.04% ) in 2.65 min. Users per second: 276
EvaluatorHoldout: Processed 52000 ( 74.50% ) in 3.19 min. Users per second: 271
EvaluatorHoldout: Processed 60000 ( 85.96% ) in 3.69 min. Users per second: 271
EvaluatorHoldout: Processed 68000 ( 97.42% ) in 4.20 min. Users per second: 270
EvaluatorHoldout: Processed 69799 ( 100.00% ) in 4.33 min. Users per second: 269
CPU times: user 6min 44s, sys: 40.8 s, total: 7min 25s
Wall time: 6min 28s


In [52]:
result_dict_puresvditem

{5: {'ROC_AUC': 0.48895280257119433,
  'PRECISION': 0.358268743105283,
  'PRECISION_RECALL_MIN_DEN': 0.36531803225458637,
  'RECALL': 0.1114512892092393,
  'MAP': 0.2903381734217877,
  'MRR': 0.5739736003858659,
  'NDCG': 0.16191103604428606,
  'F1': 0.17001409586773145,
  'HIT_RATE': 1.7913437155260103,
  'ARHR': 0.904660525222383,
  'NOVELTY': 0.004571891522989991,
  'AVERAGE_POPULARITY': 0.3546708396952197,
  'DIVERSITY_MEAN_INTER_LIST': 0.9778826132510097,
  'DIVERSITY_HERFINDAHL': 0.9955737206541468,
  'COVERAGE_ITEM': 0.08014230877258684,
  'COVERAGE_ITEM_CORRECT': 0.07733358299784665,
  'COVERAGE_USER': 0.9988694581985746,
  'COVERAGE_USER_CORRECT': 0.7528263545035634,
  'DIVERSITY_GINI': 0.020775983819095234,
  'SHANNON_ENTROPY': 8.22549615909713},
 20: {'ROC_AUC': 0.5843590001947071,
  'PRECISION': 0.23961159902002463,
  'PRECISION_RECALL_MIN_DEN': 0.3470135352495496,
  'RECALL': 0.2566079089314121,
  'MAP': 0.20103537198833224,
  'MRR': 0.5894095745398737,
  'NDCG': 0.2526918

In [53]:
%%time

recommender = MatrixFactorization_BPR_Cython(URM_train)
recommender.fit(num_factors = 50, 
                validation_every_n = 10, 
                stop_on_validation = True, 
                evaluator_object = evaluator_validation_early_stopping,
                lower_validations_allowed = 5, 
                validation_metric = "MAP")

result_dict_bpr, _ = evaluator_test.evaluateRecommender(recommender)

MatrixFactorization_BPR_Cython_Recommender: URM Detected 23 (0.22 %) cold items.
MF_BPR: Processed 69000 ( 98.57% ) in 0.80 seconds. BPR loss 1.02E-02. Sample per second: 86166
MF_BPR: Epoch 1 of 300. Elapsed time 0.52 sec
MF_BPR: Processed 69000 ( 98.57% ) in 1.31 seconds. BPR loss 1.01E-02. Sample per second: 52509
MF_BPR: Epoch 2 of 300. Elapsed time 1.04 sec
MF_BPR: Processed 69000 ( 98.57% ) in 0.85 seconds. BPR loss 1.01E-02. Sample per second: 81237
MF_BPR: Epoch 3 of 300. Elapsed time 1.57 sec
MF_BPR: Processed 69000 ( 98.57% ) in 1.48 seconds. BPR loss 1.02E-02. Sample per second: 46737
MF_BPR: Epoch 4 of 300. Elapsed time 2.20 sec
MF_BPR: Processed 69000 ( 98.57% ) in 1.03 seconds. BPR loss 1.02E-02. Sample per second: 66828
MF_BPR: Epoch 5 of 300. Elapsed time 2.75 sec
MF_BPR: Processed 69000 ( 98.57% ) in 0.54 seconds. BPR loss 1.01E-02. Sample per second: 127321
MF_BPR: Epoch 6 of 300. Elapsed time 3.26 sec
MF_BPR: Processed 69000 ( 98.57% ) in 1.03 seconds. BPR loss 1.01E

MF_BPR: Processed 69000 ( 98.57% ) in 1.47 seconds. BPR loss 1.01E-02. Sample per second: 46842
MF_BPR: Epoch 41 of 300. Elapsed time 3.29 min
MF_BPR: Processed 69000 ( 98.57% ) in 0.94 seconds. BPR loss 1.02E-02. Sample per second: 73512
MF_BPR: Epoch 42 of 300. Elapsed time 3.29 min
MF_BPR: Processed 69000 ( 98.57% ) in 1.39 seconds. BPR loss 1.02E-02. Sample per second: 49651
MF_BPR: Epoch 43 of 300. Elapsed time 3.30 min
MF_BPR: Processed 69000 ( 98.57% ) in 0.84 seconds. BPR loss 1.01E-02. Sample per second: 82294
MF_BPR: Epoch 44 of 300. Elapsed time 3.31 min
MF_BPR: Processed 69000 ( 98.57% ) in 1.28 seconds. BPR loss 1.01E-02. Sample per second: 53800
MF_BPR: Epoch 45 of 300. Elapsed time 3.32 min
MF_BPR: Processed 69000 ( 98.57% ) in 0.73 seconds. BPR loss 1.01E-02. Sample per second: 94114
MF_BPR: Epoch 46 of 300. Elapsed time 3.32 min
MF_BPR: Processed 69000 ( 98.57% ) in 1.18 seconds. BPR loss 1.03E-02. Sample per second: 58519
MF_BPR: Epoch 47 of 300. Elapsed time 3.33 min

EvaluatorHoldout: Processed 42000 ( 60.17% ) in 30.69 sec. Users per second: 1369
EvaluatorHoldout: Processed 69799 ( 100.00% ) in 50.54 sec. Users per second: 1381
CPU times: user 11min 37s, sys: 1min 12s, total: 12min 49s
Wall time: 6min 56s


In [54]:
result_dict_bpr

{5: {'ROC_AUC': 0.0066679560834204885,
  'PRECISION': 0.002727832776973851,
  'PRECISION_RECALL_MIN_DEN': 0.0027526659885289354,
  'RECALL': 0.0005234563207056631,
  'MAP': 0.0012606834545544291,
  'MRR': 0.006124490799772649,
  'NDCG': 0.0006706927944033529,
  'F1': 0.000878360100277859,
  'HIT_RATE': 0.013639163884869411,
  'ARHR': 0.0061877677330620445,
  'NOVELTY': 0.007523037993188253,
  'AVERAGE_POPULARITY': 0.025128165704133305,
  'DIVERSITY_MEAN_INTER_LIST': 0.9987433318396813,
  'DIVERSITY_HERFINDAHL': 0.9997458045981922,
  'COVERAGE_ITEM': 0.9403613893830166,
  'COVERAGE_ITEM_CORRECT': 0.05692350903473457,
  'COVERAGE_USER': 0.9988694581985746,
  'COVERAGE_USER_CORRECT': 0.013409084404247403,
  'DIVERSITY_GINI': 0.40698870372049323,
  'SHANNON_ENTROPY': 12.480280868232452},
 20: {'ROC_AUC': 0.02504328745491179,
  'PRECISION': 0.002770096992793701,
  'PRECISION_RECALL_MIN_DEN': 0.003412251399730683,
  'RECALL': 0.0019263672614757638,
  'MAP': 0.0006398198636900119,
  'MRR': 0.

In [55]:
%%time

recommender = MatrixFactorization_FunkSVD_Cython(URM_train)
recommender.fit(num_factors = 50, 
                validation_every_n = 10, 
                stop_on_validation = True, 
                evaluator_object = evaluator_validation_early_stopping,
                lower_validations_allowed = 5, 
                validation_metric = "MAP")

result_dict_funksvd, _ = evaluator_test.evaluateRecommender(recommender)

MatrixFactorization_FunkSVD_Cython_Recommender: URM Detected 23 (0.22 %) cold items.
FUNK_SVD: Processed 8002000 ( 99.99% ) in 35.92 seconds. MSE loss 1.95E+00. Sample per second: 222766
FUNK_SVD: Epoch 1 of 300. Elapsed time 34.93 sec
FUNK_SVD: Processed 8002000 ( 99.99% ) in 35.86 seconds. MSE loss 1.14E+00. Sample per second: 223136
FUNK_SVD: Epoch 2 of 300. Elapsed time 1.16 min
FUNK_SVD: Processed 8002000 ( 99.99% ) in 35.71 seconds. MSE loss 1.13E+00. Sample per second: 224065
FUNK_SVD: Epoch 3 of 300. Elapsed time 1.75 min
FUNK_SVD: Processed 8002000 ( 99.99% ) in 36.18 seconds. MSE loss 1.13E+00. Sample per second: 221168
FUNK_SVD: Epoch 4 of 300. Elapsed time 2.34 min
FUNK_SVD: Processed 8002000 ( 99.99% ) in 35.31 seconds. MSE loss 1.13E+00. Sample per second: 226616
FUNK_SVD: Epoch 5 of 300. Elapsed time 2.92 min
FUNK_SVD: Processed 8002000 ( 99.99% ) in 35.55 seconds. MSE loss 1.12E+00. Sample per second: 225073
FUNK_SVD: Epoch 6 of 300. Elapsed time 3.51 min
FUNK_SVD: Proc

FUNK_SVD: Processed 8002000 ( 99.99% ) in 34.83 seconds. MSE loss 1.06E+00. Sample per second: 229748
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 57000 ( 81.57% ) in 30.45 sec. Users per second: 1872
EvaluatorHoldout: Processed 69878 ( 100.00% ) in 37.38 sec. Users per second: 1869
FUNK_SVD: CUTOFF: 5 - ROC_AUC: 0.3832978, PRECISION: 0.3229972, PRECISION_RECALL_MIN_DEN: 0.3229972, RECALL: 0.0243043, MAP: 0.2347366, MRR: 0.4836985, NDCG: 0.0702302, F1: 0.0452070, HIT_RATE: 1.6149861, ARHR: 0.7570428, NOVELTY: 0.0039743, AVERAGE_POPULARITY: 0.8115235, DIVERSITY_MEAN_INTER_LIST: 0.3462916, DIVERSITY_HERFINDAHL: 0.8692573, COVERAGE_ITEM: 0.0036513, COVERAGE_ITEM_CORRECT: 0.0033705, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.7114256, DIVERSITY_GINI: 0.0007632, SHANNON_ENTROPY: 3.2455180, 

FUNK_SVD: Epoch 40 of 300. Elapsed time 26.87 min
FUNK_SVD: Processed 8002000 ( 99.99% ) in 34.70 seconds. MSE loss 1.06E+00. Sample per second: 230637
FUNK_SVD: Epoch 41 of 300. El

In [56]:
result_dict_funksvd

{5: {'ROC_AUC': 0.2677820121587237,
  'PRECISION': 0.14230289832231663,
  'PRECISION_RECALL_MIN_DEN': 0.14410258981742163,
  'RECALL': 0.03642031146152129,
  'MAP': 0.096818037826076,
  'MRR': 0.28266116515519757,
  'NDCG': 0.06774389907924616,
  'F1': 0.05799712175093941,
  'HIT_RATE': 0.7115144916116277,
  'ARHR': 0.3678120985496398,
  'NOVELTY': 0.004175950165821664,
  'AVERAGE_POPULARITY': 0.7515882247201602,
  'DIVERSITY_MEAN_INTER_LIST': 0.6512571033766005,
  'DIVERSITY_HERFINDAHL': 0.9302495545823865,
  'COVERAGE_ITEM': 0.46587398183690665,
  'COVERAGE_ITEM_CORRECT': 0.021439940080516806,
  'COVERAGE_USER': 0.9988694581985746,
  'COVERAGE_USER_CORRECT': 0.4445175877958728,
  'DIVERSITY_GINI': 0.013815009439239335,
  'SHANNON_ENTROPY': 4.847633844975068},
 20: {'ROC_AUC': 0.44720211062555526,
  'PRECISION': 0.06784982592875835,
  'PRECISION_RECALL_MIN_DEN': 0.09772306205830338,
  'RECALL': 0.07263442207382349,
  'MAP': 0.04279839440348588,
  'MRR': 0.30286159857056405,
  'NDCG': 

What about the performance of these recommenders?

In [57]:
# MAP@20

puresvd = result_dict_puresvd[20]['MAP']
puresvd_i = result_dict_puresvditem[20]['MAP']
bpr = result_dict_bpr[20]['MAP']
funksvd = result_dict_funksvd[20]['MAP']

print("Results for MAP")
print("PureSVD\t|PureSVD-ItemBased\t|BPRMF\t|FunkSVD")
print(f"{puresvd:.2f}\t|{puresvd_i:.2f}\t|{bpr:.2f}\t|{funksvd:.2f}")

Results for MAP
PureSVD	|PureSVD-ItemBased	|BPRMF	|FunkSVD
0.20	|0.20	|0.00	|0.04


In [58]:
# Precision@20

puresvd = result_dict_puresvd[20]['PRECISION']
puresvd_i = result_dict_puresvditem[20]['PRECISION']
bpr = result_dict_bpr[20]['PRECISION']
funksvd = result_dict_funksvd[20]['PRECISION']

print("Results for PRECISION")
print("PureSVD\t|PureSVD-ItemBased\t|BPRMF\t|FunkSVD")
print(f"{puresvd:.2f}\t|{puresvd_i:.2f}\t|{bpr:.2f}\t|{funksvd:.2f}")

Results for PRECISION
PureSVD	|PureSVD-ItemBased	|BPRMF	|FunkSVD
0.24	|0.24	|0.00	|0.07


In [59]:
# Recall@20

puresvd = result_dict_puresvd[20]['RECALL']
puresvd_i = result_dict_puresvditem[20]['RECALL']
bpr = result_dict_bpr[20]['RECALL']
funksvd = result_dict_funksvd[20]['RECALL']

print("Results for RECALL")
print("PureSVD\t|PureSVD-ItemBased\t|BPRMF\t|FunkSVD")
print(f"{puresvd:.2f}\t|{puresvd_i:.2f}\t|{bpr:.2f}\t|{funksvd:.2f}")

Results for RECALL
PureSVD	|PureSVD-ItemBased	|BPRMF	|FunkSVD
0.26	|0.26	|0.00	|0.07


## Extra

This section is for you to practice and analyze different aspects of what you saw in the notebook.

1. We briefly mentioned early stopping. Research on how to use it and when to use it. Implement it in models that might require it.

2. The comparison we did was not fair in terms of parameters and their tuning. Run a hyperparameter tuning of the algorithms presented in the notebook and compare the best performant ones.

3. Read about these other Matrix Factorization techniques:
 * Non Negative Matrix Factorization Recommender (NMFRecommender)
 * Binary/Implicit Alternating Least Squares (IALS) (IALSRecommender in the material)
 
 Familiarize yourself with these and compare them with what you've already know (BPR MF, FunkSVD, PureSVD, MF with PyTorch). What are the differences and similarities with what you've already seen?

